<img src="https://avatars.githubusercontent.com/u/74911464?s=200&v=4"
     alt="OpenEO Platform logo"
     style="float: left; margin-right: 10px;" />
# openEO Platform UC8

### Requirements

In [1]:
import openeo
from openeo.processes import median, mean
from openeo.rest.datacube import PGNode, THIS
import geopandas as gpd

## Connection

Connect to the OpenEO back-end using the OpenEO client

In [2]:
# conn = openeo.connect("openeo.cloud")
conn = openeo.connect("https://openeo-dev.eodc.eu/v1.0")

Authenticate via EGI Check-in (OpenID Connect)

In [3]:
conn.authenticate_oidc("egi")

Authenticated using refresh token.


<Connection to 'https://openeo-dev.eodc.eu/v1.0' with BearerAuth>

Load the polygons from the first AOI and perform an aggregation over Sentinel-2 and Sentinel-1

In [4]:
aoi0_gdf = gpd.read_file("https://raw.githubusercontent.com/openEOPlatform/SRR3_notebooks/main/notebooks/resources/UC8/vector_data/target_canopy_cover_60m_equi7/target_canopy_cover_equi7_60m_AOI0.geojson")
aoi0_gdf

,id,target_canopy_cover,geometry
0,0,0.118795,"POLYGON ((4691600.000 1950570.000, 4691660.000..."
1,1,0.699634,"POLYGON ((4691600.000 1950510.000, 4691660.000..."
2,2,1.000000,"POLYGON ((4691660.000 1950870.000, 4691720.000..."
3,3,0.833854,"POLYGON ((4691660.000 1950810.000, 4691720.000..."
4,4,0.119751,"POLYGON ((4691660.000 1950750.000, 4691720.000..."
5,5,0.016959,"POLYGON ((4691660.000 1950690.000, 4691720.000..."
6,6,0.000000,"POLYGON ((4691660.000 1950630.000, 4691720.000..."
7,7,0.034963,"POLYGON ((4691660.000 1950570.000, 4691720.000..."
8,8,0.113942,"POLYGON ((4691660.000 1950510.000, 4691720.000..."
9,9,1.000000,"POLYGON ((4691720.000 1950870.000, 4691780.000..."


Get the AOI bounds in EQUI7 and use them in load_collection.

In [5]:
aoi_bounds = aoi0_gdf.total_bounds
aoi_bounds

array([4691600., 1950510., 4692080., 1950930.])

Load the Sentinel-2 data for summer 2018 and take the median over time:

In [6]:
collection      = "boa_sentinel_2"
spatial_extent  = {"crs": "PROJCRS[\"Azimuthal_Equidistant\",BASEGEOGCRS[\"WGS 84\",DATUM[\"World Geodetic System 1984\",ELLIPSOID[\"WGS 84\",6378137,298.257223563,LENGTHUNIT[\"metre\",1]]],PRIMEM[\"Greenwich\",0,ANGLEUNIT[\"degree\",0.0174532925199433]],ID[\"EPSG\",4326]],CONVERSION[\"Modified Azimuthal Equidistant\",METHOD[\"Modified Azimuthal Equidistant\",ID[\"EPSG\",9832]],PARAMETER[\"Latitude of natural origin\",53,ANGLEUNIT[\"degree\",0.0174532925199433],ID[\"EPSG\",8801]],PARAMETER[\"Longitude of natural origin\",24,ANGLEUNIT[\"degree\",0.0174532925199433],ID[\"EPSG\",8802]],PARAMETER[\"False easting\",5837287.81977,LENGTHUNIT[\"metre\",1],ID[\"EPSG\",8806]],PARAMETER[\"False northing\",2121415.69617,LENGTHUNIT[\"metre\",1],ID[\"EPSG\",8807]]],CS[Cartesian,2],AXIS[\"easting\",east,ORDER[1],LENGTHUNIT[\"metre\",1,ID[\"EPSG\",9001]]],AXIS[\"northing\",north,ORDER[2],LENGTHUNIT[\"metre\",1,ID[\"EPSG\",9001]]]]",
                   "east": aoi_bounds[0],
                   "north": aoi_bounds[1],
                   "south": aoi_bounds[3],
                   "west": aoi_bounds[2]}
temporal_extent = ["2018-05-01", "2018-09-01"]
bands           = ["B02","B03","B04","B08"]

boa_sentinel_2_cube = conn.load_collection(
    collection_id   = collection,
    spatial_extent  = spatial_extent,
    temporal_extent = temporal_extent,
    bands = bands
    )

In [7]:
boa_sentinel_2_cube_reduced = boa_sentinel_2_cube.reduce_dimension(dimension="t",reducer=median)

Load the Sentinel-1 data for summer 2018 and take the median over time:

In [8]:
collection      = "SIG0_Sentinel_1"
spatial_extent  = {"crs": "PROJCRS[\"Azimuthal_Equidistant\",BASEGEOGCRS[\"WGS 84\",DATUM[\"World Geodetic System 1984\",ELLIPSOID[\"WGS 84\",6378137,298.257223563,LENGTHUNIT[\"metre\",1]]],PRIMEM[\"Greenwich\",0,ANGLEUNIT[\"degree\",0.0174532925199433]],ID[\"EPSG\",4326]],CONVERSION[\"Modified Azimuthal Equidistant\",METHOD[\"Modified Azimuthal Equidistant\",ID[\"EPSG\",9832]],PARAMETER[\"Latitude of natural origin\",53,ANGLEUNIT[\"degree\",0.0174532925199433],ID[\"EPSG\",8801]],PARAMETER[\"Longitude of natural origin\",24,ANGLEUNIT[\"degree\",0.0174532925199433],ID[\"EPSG\",8802]],PARAMETER[\"False easting\",5837287.81977,LENGTHUNIT[\"metre\",1],ID[\"EPSG\",8806]],PARAMETER[\"False northing\",2121415.69617,LENGTHUNIT[\"metre\",1],ID[\"EPSG\",8807]]],CS[Cartesian,2],AXIS[\"easting\",east,ORDER[1],LENGTHUNIT[\"metre\",1,ID[\"EPSG\",9001]]],AXIS[\"northing\",north,ORDER[2],LENGTHUNIT[\"metre\",1,ID[\"EPSG\",9001]]]]",
                   "east": aoi_bounds[0],
                   "north": aoi_bounds[1],
                   "south": aoi_bounds[3],
                   "west": aoi_bounds[2]}
temporal_extent = ["2018-05-01", "2018-09-01"]
bands           = ["vh","vv"]

sigma0_sentinel_1_cube = conn.load_collection(
    collection_id   = collection,
    spatial_extent  = spatial_extent,
    temporal_extent = temporal_extent,
    bands = bands
    )

In [9]:
sigma0_sentinel_1_cube_reduced = sigma0_sentinel_1_cube.reduce_dimension(dimension="t",reducer=median)

Reproject Sentinel-1 20m to match the Sentinel-2 data in 10m using `resample_cube_spatial`:

In [10]:
sigma0_sentinel_1_cube_reduced_10m = sigma0_sentinel_1_cube_reduced.resample_cube_spatial(target=boa_sentinel_2_cube_reduced)

Merge Sentinel-2 and Sentinel-1 into a single datacube

In [11]:
sentinel_2_and_1 = sigma0_sentinel_1_cube_reduced_10m.merge_cubes(boa_sentinel_2_cube_reduced)

Now that we have a single datacube with the Sentinel-2 and Sentinel-1 bands, we can perform our spatial aggregation to generate our training dataset.

We firstly load our vector-cube defining the polygons with associated target canopy cover values:

In [12]:
aoi0_vector_cube = PGNode("load_vector_cube",{"URL":"https://raw.githubusercontent.com/openEOPlatform/SRR3_notebooks/main/notebooks/resources/UC8/vector_data/target_canopy_cover_60m_equi7/target_canopy_cover_equi7_60m_AOI0.geojson"})

The next step creates the predictors associated with the provided `target_canopy_cover` values in the input geoJSON.

In [13]:
sentinel_2_and_1_predictors = sentinel_2_and_1.process("aggregate_spatial",{"data":THIS,"target_dimension":"result","geometries":aoi0_vector_cube,"reducer":"mean"})

Finally, we can train our Random Forest Regression:

In [14]:
rf_arguments = {
    "predictors": sentinel_2_and_1_predictors,
    "target": aoi0_vector_cube,
    "training": 1.0,
    "max_variables" : "all",
    "num_trees": 100,
    "seed": 0,
    "predictors_vars": ["B02","B03","B04","B08","vv","vh"],
    "target_var": "target_canopy_cover"

}

rf_model = sentinel_2_and_1_predictors.process("fit_regr_random_forest",rf_arguments)

The model is stored using the `save_ml_model` process and can be used later for the predictions step:

In [15]:
rf_model_saved = rf_model.process("save_ml_model",{"model":THIS})

In [ ]:
job = rf_model_saved.send_job(title="UC8_AOI0_fit_rf_regr")
job_id = job.job_id
if job_id:
    print("Batch job created with id: ",job_id)
    job.start_job()
else:
    print("Error! Job ID is None")